<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании. Разработать ***метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.*** Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбор наилучшей модели не производится.

Описание данных
<br>Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
<br>***Целевой признак: количество страховых выплат клиенту за последние 5 лет.***

## Загрузка данных

In [1]:
# Загрузка библиотек
import numpy as np
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Путь к файлам
data=pd.read_csv('/datasets/insurance.csv') 
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
# Проверка типов данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
#замена типа данных с float64 на int64 
columns = ['Возраст', 'Зарплата']
for i in columns:
    data[i] = data[i].astype('int64') 

In [5]:
#Проверка наличия пропусков
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
#Проверка наличия дубликатов
data.duplicated().sum()

153

Данные загружены и готовы для построения и обоснования алгоритма защиты данных клиентов; без пропусков; 153 дубликата- незначительное количество, не трогаем; проведена замена типа данных на 'int' по столбцам 'Возраст' и 'Зарплата' для логичного отображения информации.

## Умножение матриц

В данном разделе отвечаем на вопрос:
<br>Изменится ли качество линейной регрессии (возможно обучение заново) при умножении признаков на обратимую матрицу.
<br>a. Изменится. Примеры матриц.
<br>b. Не изменится. Указываем, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении признаков на обратимую матрицу предсказания не меняются следовательно и не меняется качество линейной регрессии.

**Обоснование:** 
$$
a = X_1w
$$

$$
a_1 = X_1(X^TX)^{-1}X^Ty = X_1X^{-1}(X^T)^{-1}X^Ty=X_1X^{-1}y
$$

$$
a_2 = (X_1A)((XA)^TXA)^{-1}(XA)^Ty = (X_1A)(XA)^{-1}((XA)^T)^{-1}(XA)^Ty = (X_1A)A^{-1}X^{-1}(X^T)^{-1}(A^T)^{-1}A^TX^Ty = X_1AA^{-1}X^{-1}(X^T)^{-1}(A^T)^{-1}A^TX^Ty = X_1X^{-1}y
$$

$$
a_1=a_2
$$

В предсказание $a = Xw$ подставили значение формулы обучения- отражено в $a_1$, $a_2$- это аналогичная подстановка, но домноженная на обратимую матрицу A. 
<br>Через матрицу предсказания и векторы весов, применение свойств транспонированной матрицы выведена идентичность предсказаний $a_1$ и $a_2$.

## Алгоритм преобразования

Предлагаем алгоритм преобразования данных для решения задачи защиты данных клиентов страховой компании.

**Алгоритм**

1. Определение целевого признака и подготовка всех признаков к обучению;
2. Обучение модели, выход по метрике R2;
3. Создание обратимой матрицы и домножение на матрицу признаков;
4. Обучение модели с преобразованными признаками, выход по метрике R2(inv);
5. Сравнение двух метрик (пункты 2 и 4).

**Обоснование**

Обратная матрица A существует только для квадратных невырожденных матриц определитель которых не равен нулю.
<br>Приведенный расчет $a_1$ и $a_2$ показывает идентичность размерности матриц признаков и обратимой.

## Проверка алгоритма

Обосновываем, почему качество линейной регрессии не поменяется.
<br>Программируем алгоритм, применив матричные операции. 
<br>Проверяем, что качество линейной регрессии из sklearn не отличается до и после преобразования метрикой R2.

In [7]:
# Выделение целевого признака
X = data.drop('Страховые выплаты', axis=1)
y = data['Страховые выплаты']
X.shape, y.shape

((5000, 4), (5000,))

In [8]:
# Обучение модели
def r2_score(X, y):
    model_lr = LinearRegression()
    model_lr.fit(X, y)
    R2 = model_lr.score(X, y)
    return R2

In [9]:
# Расчет метрики R2 для матрицы признаков
R2 = r2_score(X, y)
print(("R2 = {0:.3f}".format(R2)))

R2 = 0.425


In [10]:
# Создание случайной обратимой матрицы
inv = np.random.rand(4,4)
inv

array([[4.76305772e-01, 4.69357908e-01, 7.35062580e-01, 3.30699475e-01],
       [2.35233470e-01, 8.27667316e-01, 1.97005895e-03, 5.74456642e-02],
       [6.66667447e-01, 4.71977600e-01, 7.88165965e-02, 3.60260326e-01],
       [2.89998872e-01, 6.73308112e-04, 5.45564262e-01, 6.16874113e-01]])

In [11]:
# Домножение матрицы признаков на обратимую
X_inv = X @ inv
X_inv.head()

,0,1,2,3
0,33077.116253,23444.493326,3910.664588,17872.214999
1,25344.473728,17973.222151,2995.666856,13693.151750
2,14006.838160,9935.531942,1655.205659,7567.132763
3,27805.552445,19698.848259,3287.784576,15025.295687
4,17407.083212,12342.259390,2057.903394,9404.733678


In [12]:
# Расчет метрики R2 для преобразованной матрицы
R2_inv = r2_score(X_inv, y)
print(("R2_inv = {0:.3f}".format(R2_inv)))

R2_inv = 0.425


## Вывод:
Использование матрицы признаков и аналогичной умноженной на обратимую не изменило ее предсказания и качество линейной регрессии; идентичные значение R2= 0.425. 
<br>Сформирован и проверен алгоритм преобразования данных.
<br>Итогом работы является понимание возможности защиты персональной информации используя правильные матрицы.